In [27]:
import pandas as pd
from numpy import array
import tensorflow
from tensorflow import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [31]:
import data_preprocessing
from data_preprocessing import*

trainPath='C:/Aarohi DataFiles/Studies/MS/03 - Sem 3/CSCI 6364 - Machine Learning/Final Project/liar_dataset/train.tsv'
testPath='C:/Aarohi DataFiles/Studies/MS/03 - Sem 3/CSCI 6364 - Machine Learning/Final Project/liar_dataset/test.tsv'
validationPath='C:/Aarohi DataFiles/Studies/MS/03 - Sem 3/CSCI 6364 - Machine Learning/Final Project/liar_dataset/test.tsv'

trainX,trainY,testX,testY,valX,valY=data_preprocess(trainPath,testPath,validationPath,pred="single")

In [39]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [40]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
 
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 

 
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
 
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0
 
 
sentiment=[]
X = []  
sentences = list(hate_comment["the_statement"])
for i in range(len(sentences)):
    sen=preprocess_text(sentences[i])
    if swn_polarity(sen)==1:
        X.append(sen+" positive")
        sentiment.append(1)
    else:
        X.append(sen+" negative")
        sentiment.append(0)
    


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [42]:
#sentiment matrix with target liar labels
#todo

y

array([0, 0, 1, ..., 0, 0, 0])

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
def get_accuracy(review,sentiment,model,lr,show_coefficients=False):
    review_model=model.fit_transform(review)
    test_model=model.transform(testX)
    #review_train,review_test,target_train,target_test=train_test_split(review_model,sentiment,test_size=0.2,random_state=0)
    final_model=lr.fit(review_model,sentiment)
    accuracy=final_model.score(test_model,testY)
    print("Accuracy is "+str(accuracy))
    if show_coefficients:
        df=pd.DataFrame({'Word':model.get_feature_names(),'Coefficient':final_model.coef_.tolist()[0]}).sort_values(['Coefficient','Word'],ascending=[0,1])
        print("-----------------Top 25 positive words------------")
        print(df.head(10).to_string(index=False))
        print("-----------------Top 25 negative words------------")
        print(df.tail(10).to_string(index=False))

In [58]:
get_accuracy(trainX,trainY,TfidfVectorizer(ngram_range=(1,2),max_features=300),LogisticRegression(max_iter=300),show_coefficients=True)

Accuracy is 0.7971864009378663
-----------------Top 25 positive words------------
       Word  Coefficient
      today     1.601055
    highest     1.559768
     at the     1.235394
      month     1.196493
        got     1.123119
     immigr     1.098391
       wage     1.039887
 in america     1.005510
       half     0.926486
     of the     0.924621
-----------------Top 25 negative words------------
     Word  Coefficient
       on    -0.851064
     will    -0.902373
  say the    -0.906583
     know    -0.944415
      war    -0.971157
       if    -1.021704
  medicar    -1.034999
 has been    -1.328472
    abort    -1.378694
     your    -1.418423


In [63]:
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


# estimate bias and variance
model = TfidfVectorizer(ngram_range=(1,2),max_features=400)
lr = LogisticRegression(max_iter=300)
review_model=model.fit_transform(trainX)
test_model = model.transform(testX)
#print(review_model)
#review_train,review_test,target_train,target_test=train_test_split(review_model,y,test_size=0.2,random_state=0)
final_model=lr.fit(review_model,trainY)
print(final_model)
mse, bias, var = bias_variance_decomp(final_model, review_model, trainY, test_model, testY, loss='mse', num_rounds=200, random_seed=1)
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)

LogisticRegression(max_iter=300)
MSE: 0.211
Bias: 0.201
Variance: 0.010


In [68]:
import xgboost

# Xgb boost Classifier to reduce bias

print("=====xgb boost classifier=====")

model = TfidfVectorizer(ngram_range=(1,2),max_features=300,binary=True)
review_model=model.fit_transform(X)
test_model = model.transform(testX)

#review_train,review_test,target_train,target_test=train_test_split(review_model,y,test_size=0.2,random_state=0)
xgb = xgboost.XGBClassifier()
final_model = xgb.fit(review_model, trainY)
rf_results = xgb.predict(test_model)
accuracy=final_model.score(test_model,testY)
print("Accuracy is "+str(accuracy))

=====xgb boost classifier=====
Accuracy is 0.7749120750293084


In [69]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
              "max_depth"        : [3, 4, 5, 6, 8, 10, 12, 15],
              "min_child_weight" : [1, 3, 5, 7],
              "gamma"            : [0.0, 0.1, 0.2 , 0.3, 0.4],
              "colsample_bytree" : [0.3, 0.4, 0.5 , 0.7]}

randomXgb = RandomizedSearchCV(estimator=xgb,
                               param_distributions=param_grid,
                               n_iter=20,
                               n_jobs=-1,
                               verbose=2,
                               cv=3,
                               random_state=42)

In [70]:
finalXGB = randomXgb.estimator

print(randomXgb.get_params)

<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_delta_step=0, max_depth=6,
                                           min_child_weight=1, missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=100, n_jobs=0,
                                           num_pa...
                                           reg_alpha=0, reg_lambda=1,
                        

In [71]:
# For liar dataset
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred_xgb = finalXGB.predict(test_model)

print("Accuracy : {} %".format(round(accuracy_score(testY, y_pred_xgb)*100, 4)))

Accuracy : 77.4912 %


In [72]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Gradient Boosting Classifier

print("=====Gradient Boosting Classifier=====")

model = TfidfVectorizer(ngram_range=(1,2),max_features=300,binary=True)
review_model=model.fit_transform(X)
test_model = model.transform(testX)

#review_train,review_test,target_train,target_test=train_test_split(review_model,y,test_size=0.2,random_state=0)

gbc = GradientBoostingClassifier(random_state=0)
final_model = gbc.fit(review_model, trainY)
gbc_results = gbc.predict(test_model)
accuracy=final_model.score(test_model,testY)
print("Accuracy is "+str(accuracy))

=====Gradient Boosting Classifier=====
Accuracy is 0.794841735052755


In [73]:
# Bias and variance after boosting
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


# estimate bias and variance
model = TfidfVectorizer(ngram_range=(1,2),max_features=400)
lr = LogisticRegression(max_iter=300)
review_model=model.fit_transform(trainX)
test_model = model.transform(testX)
#print(review_model)
#review_train,review_test,target_train,target_test=train_test_split(review_model,y,test_size=0.2,random_state=0)
final_model=lr.fit(review_model,trainY)
print(final_model)
mse, bias, var = bias_variance_decomp(final_model, review_model, trainY, test_model, testY, loss='mse', num_rounds=200, random_seed=1)
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)

LogisticRegression(max_iter=300)
MSE: 0.211
Bias: 0.201
Variance: 0.010
